## 1. 웹스크래핑 연습문제

* user-agent 요청헤더를 반드시 설정해야 한다.

In [ ]:
# requests 라이브러리 설치여부 확인
!pip show requests
# beautifulsoup4 라이브러리 설치여부 확인
!pip show beautifulsoup4

In [2]:
# reqeusts, bs4 import
import requests
import bs4
# BeautifulSoup 클래스 import
from bs4 import BeautifulSoup

### 질문 1: Daum 뉴스기사 제목 스크래핑하기

In [ ]:
# 다음 경제 뉴스 URL
url = 'https://news.daum.net/economy'
print(url)

# 요청 헤더 설정 : 브라우저 정보
req_header = {
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/144.0.0.0 Safari/537.36'
}

# requests 의 get() 함수 호출하기 
res = requests.get(url, headers=req_header)
print(type(res))
print(res.status_code)
# 응답(response)이 OK 이면 응답 (response)에서 text 추출
# BeautifulSoup 객체 생성
if res.ok :
    res.encoding = 'utf-8'
    soup = BeautifulSoup(res.text, 'html.parser')
    # CSS 선택자를 사용해서 a tag 목록 가져오기
    a_tags = soup.select("ul.list_newsheadline2 a[href*='https://v.daum.net/v/']")
    print(type(a_tags), len(a_tags))
    # <a> 태그 리스트 순회하기    
    for a_tag in a_tags :
        link = a_tag['href']
        strong_tag = a_tag.select_one('div.cont_thumb strong.tit_txt')
        title = strong_tag.text
        
        print(link, title)  
else :
    # 응답(response)이 Error 이면 status code 출력    
    print(f'Error Code = {res.status_code}')

### 질문2:  여러개의 section 중 하나를 선택해서 url에서 뉴스기사의 링크와 제목을 출력하는 코드를 함수로 작성하기

In [ ]:
import requests
from bs4 import BeautifulSoup

section_dict = {'기후/환경':'climate','사회':'society','경제':'economy','정치':'politics',\
             '국제':'world','문화':'culture','생활':'life','IT/과학':'tech','인물':'people'}

def print_news(section_name):  #print_new(life,'생활') 
    sid = section_dict.get(section_name)
    url = f'https://news.daum.net/{sid}'
    print(f'======> {url} {section_name} 뉴스 <======')
    req_header = {
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/144.0.0.0 Safari/537.36'
}
    res = requests.get(url, headers=req_header)
    if res.ok :
        res.encoding = 'utf-8'
        soup = BeautifulSoup(res.text, 'html.parser')
        # CSS 선택자를 사용해서 a tag 목록 가져오기
        a_tags = soup.select("ul.list_newsheadline2 a[href*='https://v.daum.net/v/']")
        # <a> 태그 리스트 순회하기    
        for a_tag in a_tags :
            link = a_tag['href']
            strong_tag = a_tag.select_one('div.cont_thumb strong.tit_txt')
            title = strong_tag.text
            
            print(link)
            print(title)
    else :
        # 응답(response)이 Error 이면 status code 출력    
        print(f'Error Code = {res.status_code}')

print_news('경제')

### 2-1. Nate 뉴스기사 제목 스크래핑하기

In [ ]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from IPython.display import Image, display

section_dict = { '최신 뉴스': 'n0100', '정치': 'n0200', '경제': 'n0300', '사회': 'n0400', '세계': 'n0500', 'IT/과학': 'n0600'
}

def nate_news_scrape(section_name):
    sid = section_dict.get(section_name)
    url = f'https://news.nate.com/recent?mid={sid}'
    req_header = {
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/144.0.0.0 Safari/537.36'
    }
    
    res = requests.get(url, headers=req_header)
    
    if res.ok:
        soup = BeautifulSoup(res.text, 'html.parser')
        # 뉴스 기사 div 태그 선택 (각 뉴스 기사 단위)
        articles = soup.select('div.mduSubjectList div.mlt01')

        for article in articles[:5]:
            # img 엘리먼트 존재 여부 체크
            img_tag = article.select_one('img')
            if img_tag and img_tag.get('src'):
                img_url = urljoin(url, img_tag['src'])
                display(Image(url=img_url))  # 이미지 출력
            else:
                print('이미지 없음')

            # 기사 제목 + 기사 링크
            a_tag = article.select_one('a')
            title_tag = a_tag.select_one('h2')
            title = title_tag.text.strip()
            link = urljoin(url, a_tag['href'])
            print('기사제목:', title)
            print('기사링크:', link)

    else:
        # 응답(response)이 Error 이면 status code 출력  
        print(f'Error Code = {res.status_code}')

nate_news_scrape('최신 뉴스')

기사제목: 22세 피겨 천재 말리닌, 50년 만에 '백플립' 해금…올림픽 빙판 위 새역사
기사링크: https://news.nate.com/view/20260208n08495?mid=n0100


기사제목: [포토] 눈 덮인 도쿄 도심…은빛으로 물든 겨울 일상
기사링크: https://news.nate.com/view/20260208n08494?mid=n0100


기사제목: [포토]설 앞둔 재래시장, 북적북적
기사링크: https://news.nate.com/view/20260208n08492?mid=n0100


기사제목: 조국 최후통첩 "민주당, 13일까지 공식 입장 없으면 합당 없다"
기사링크: https://news.nate.com/view/20260208n07457?mid=n0100


기사제목: "인맥 축구다" 비판 들었던 에반스 코치, 토트넘전 '세트피스 골' 만든 주인공
기사링크: https://news.nate.com/view/20260208n08490?mid=n0100


* 현재 요청된 페이지의 image 모두 다운로드 해보기

In [17]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from IPython.display import Image, display

# 1. 섹션 정보 설정
section_dict = { 
    '최신 뉴스': 'n0100', '정치': 'n0200', '경제': 'n0300', 
    '사회': 'n0400', '세계': 'n0500', 'IT/과학': 'n0600'
}

def nate_news_scrape(section_name):
    sid = section_dict.get(section_name)
    if not sid:
        print(f"[{section_name}] 은(는) 잘못된 섹션 이름입니다.")
        return

    url = f'https://news.nate.com/recent?mid={sid}'
    
    # 서버 차단을 방지하기 위한 User-Agent 설정
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
    }
    
    res = requests.get(url, headers=headers)
    soup = BeautifulSoup(res.text, "html.parser")

    # 뉴스 리스트 아이템 선택 (이미지, 제목, 링크를 포함하는 컨테이너)
    # 네이트 최신뉴스는 보통 .mlt01 클래스를 가진 div 내에 위치합니다.
    articles = soup.select(".mlt01")

    print(f"\n{'='*50}\n[ {section_name} ] 섹션 뉴스 스크래핑 결과\n{'='*50}")

    for article in articles:
        # 기사 제목 및 링크 추출
        title_tag = article.select_one(".lt1 > a")
        if not title_tag: continue
        
        title = title_tag.get_text(strip=True)
        link = urljoin(url, title_tag.get("href"))

        # 3) img 엘리먼트 존재 여부 체크 후 이미지 추출
        img_tag = article.select_one("img")
        
        print(f"\n제목: {title}")
        print(f"링크: {link}")

        if img_tag and img_tag.get("src"):
            img_src = img_tag["src"]
            # 1) urljoin을 사용하여 도메인과 경로 합치기
            img_url = urljoin(url, img_src)
            
            # 2) Image 클래스와 display 함수를 사용하여 출력
            try:
                display(Image(url=img_url))
            except:
                print("이미지를 불러올 수 없습니다.")
        else:
            print("이미지 없음")
        
        print("-" * 30)

# 6개 섹션 전체 실행

nate_news_scrape('경제')


[ 경제 ] 섹션 뉴스 스크래핑 결과


### 3. 파일 업로드 하기
* http://httpbin.org/post 업로드 요청을 할 수 있는 url

In [ ]:
import requests

upload_files = {
    
}
#print(upload_files)

url = 'http://httpbin.org/post'
# file 업로드 하려면 requests의 post 함수에 files 속성을 사용합니다.



200


### 4. 캡챠(이미지) API 호출하기
* urllib 사용
* 1. 캡차 키 발급 요청
  2. 캡차 이미지 요청
  3. 사용자 입력값 검증 요청

In [ ]:
# 캡차 키 발급 요청


In [ ]:
# 캡차 이미지 요청


* requests를 사용하는 코드로 변경하기
* [requests docs](https://requests.readthedocs.io/en/latest/user/quickstart/)

In [ ]:
# 사용자 입력값 검증 요청



Error Code: 403


In [ ]:
import requests
import pprint

headers = {
    'X-Naver-Client-Id': '',
    'X-Naver-Client-Secret': '',
}

payload = {
    'query': '파이썬',
    'display': 100,
    'sort': 'sim'
}

url = 'https://openapi.naver.com/v1/search/blog.json'


# requests get(url, params, headers) 요청 

# json() 함수로 응답 결과 가져오오기
# 'title' , 'bloggername' , 'description' , 'bloggerlink' , 'link'

# 'data/nhnblog.txt' 파일 생성하기
